In [38]:
import cv2 
import numpy as np

In [48]:
import cv2
import numpy as np

source = './walking5.mp4'

cap = cv2.VideoCapture(source)

# Create old frame
_, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cap.set(cv2.CAP_PROP_FPS,10) 

# Lucas kanade params
lk_params = dict(winSize = (25, 25),
maxLevel = 4,
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Mouse function
def select_point(event, x, y, flags, params):
    global point, point_selected, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_points = np.array([[x, y]], dtype=np.float32)

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)

point_selected = False
point = ()
old_points = np.array([[]])

while True:
    _, frame = cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if point_selected is True:
#         cv2.circle(frame, point, 50, (0, 0, 255), 2)

        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_params)
        old_gray = gray_frame.copy()
        old_points = new_points

        x, y = new_points.ravel() 
        cv2.circle(frame, (x, y), 50, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(60)
    if key == 27:
        break


cap.release()
cv2.destroyAllWindows()